In [24]:
import aocd
import re
from dotenv import load_dotenv
import numpy as np
from tqdm import tqdm
import networkx as nx

load_dotenv()

puzzle = aocd.get_puzzle(day=20, year=2024)
text = puzzle.input_data
examples = puzzle.examples
print(examples)

lines = text.splitlines()

example = examples[0].input_data.splitlines()

examples

[Example(input_data='###############\n#...#...#.....#\n#.#.#.#.#.###.#\n#S#...#.#.#...#\n#######.#.#.###\n#######.#.#...#\n#######.#.###.#\n###..E#...#...#\n###.#######.###\n#...###...#...#\n#.#####.#.###.#\n#.#...#.#.#...#\n#.#.#.#.#.#.###\n#...#...#...###\n###############', answer_a='###############', answer_b=None, extra=None)]


[Example(input_data='###############\n#...#...#.....#\n#.#.#.#.#.###.#\n#S#...#.#.#...#\n#######.#.#.###\n#######.#.#...#\n#######.#.###.#\n###..E#...#...#\n###.#######.###\n#...###...#...#\n#.#####.#.###.#\n#.#...#.#.#...#\n#.#.#.#.#.#.###\n#...#...#...###\n###############', answer_a='###############', answer_b=None, extra=None)]

In [21]:
text = examples[0].input_data
lines = text.splitlines()

text

'###############\n#...#...#.....#\n#.#.#.#.#.###.#\n#S#...#.#.#...#\n#######.#.#.###\n#######.#.#...#\n#######.#.###.#\n###..E#...#...#\n###.#######.###\n#...###...#...#\n#.#####.#.###.#\n#.#...#.#.#...#\n#.#.#.#.#.#.###\n#...#...#...###\n###############'

In [25]:
arr = np.array([list(ll) for ll in lines])
yy, xx  = np.meshgrid(range(len(arr)), range(len(arr[0])))



g = nx.Graph()

for i in range(arr.shape[0]):
    for j in range(arr.shape[1]):
        if arr[i, j] in ".SE":
            g.add_node((i, j))

def is_valid(arr, n):
    return 0 <= n[0] < arr.shape[0] and 0 <= n[1] < arr.shape[1] 

def is_path(arr, n):
    return is_valid(arr, n) and arr[*n] in ".SE"

def is_wall(arr, n):
    return is_valid(arr,  n) and arr[*n] == "#"


cand_cheats = set()
for i, j in list(g.nodes):
    for d in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
        n = (i + d[0], j + d[1])
        if is_path(arr, n):
            g.add_edge((i, j), n)

        dist = (np.abs(xx - i) + np.abs(yy - j)) 
        dist_t = dist <= 20
        path = (arr == ".") | (arr == "S") | (arr == "E")

        pos = np.where((dist_t & path))
        for k, l in zip(*pos):
            cand_cheats.add(tuple(sorted(((i,j), (k,l)))))

        # n2 = (n[0] + d[0], n[1] + d[1])
        # if is_wall(arr, n) and is_path(arr, n2):
        #     cand_cheats.add(tuple(sorted(((i,j),  n2))))

cand_cheat_dist = [np.abs(i-k) + np.abs(j-l) for ((i,j), (k,l)) in cand_cheats]



print(len(cand_cheats))


t = np.where(arr == "S")
s = (int(t[0][0]), int(t[1][0]))
t = np.where(arr == "E")
e = (int(t[0][0]), int(t[1][0]))

L = nx.shortest_path_length(g, s, e)


ds = nx.shortest_path_length(g, source=s)
de = nx.shortest_path_length(g, source=e)

cheats = [L - min(ds[c[0]]+ de[c[1]], ds[c[1]] + de[c[0]])-d for c, d in zip(cand_cheats, cand_cheat_dist)]

from collections import Counter


s = 0
for k,v in sorted(Counter(cheats).items()):
    if k >= 100:
        s += v
    print(f"{v} cheats with {k} steps")

print(s)


1735061
85094 cheats with 0 steps
17011 cheats with 2 steps
42343 cheats with 4 steps
15274 cheats with 6 steps
32537 cheats with 8 steps
13931 cheats with 10 steps
28081 cheats with 12 steps
12771 cheats with 14 steps
24761 cheats with 16 steps
11962 cheats with 18 steps
23171 cheats with 20 steps
11367 cheats with 22 steps
22336 cheats with 24 steps
10848 cheats with 26 steps
20289 cheats with 28 steps
10263 cheats with 30 steps
19495 cheats with 32 steps
9752 cheats with 34 steps
18204 cheats with 36 steps
9209 cheats with 38 steps
16964 cheats with 40 steps
8802 cheats with 42 steps
15682 cheats with 44 steps
8305 cheats with 46 steps
15385 cheats with 48 steps
7844 cheats with 50 steps
13335 cheats with 52 steps
7620 cheats with 54 steps
13971 cheats with 56 steps
7365 cheats with 58 steps
12361 cheats with 60 steps
7105 cheats with 62 steps
12417 cheats with 64 steps
6892 cheats with 66 steps
11918 cheats with 68 steps
6727 cheats with 70 steps
11417 cheats with 72 steps
6473 che

In [26]:
aocd.submit(s, day=20, year=2024)

answer a: 1389
submitting for part b (part a is already completed)


That's the right answer!  You are one gold star closer to finding the Chief Historian.You have completed Day 20! You can [Shareon
  Bluesky
Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


Gtk-Message: 00:36:41.897: Not loading module "atk-bridge": The functionality is provided by GTK natively. Please try to not load it.


In [34]:
from collections import Counter

path = nx.shortest_path(g, s, e)
L = len(path)
cheats = []
for i, p in enumerate(path):
    cur_l = L- i
    for d in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
        h= g.copy()
        n1 = (p[0] + d[0], p[1] + d[1])
        n2 = (p[0] + 2*d[0], p[1] + 2*d[1])

        if not(0 <= n1[0] < arr.shape[0] and 0 <= n1[1] < arr.shape[1] and 0 <= n2[0] < arr.shape[0] and 0 <= n2[1] < arr.shape[1]):
            continue


        if arr[*n1] == "#" and arr[*n2] in ".SE":
            h.add_edge(p, n1)
            h.add_edge(n1, n2)

        actual_len = len(nx.shortest_path(h, s, e))
        cheats.append(L-actual_len)

for k,v in sorted(Counter(cheats).items(), key=lambda x: x[1]):
    print(k, v)

            

    

15 2
25 2
13 2
6 2
4 2
2 3
1 19
0 86


In [ ]:
arr = np.array([list(ll) for ll in lines])

g = nx.Graph()

for i in range(arr.shape[0]):
    for j in range(arr.shape[1]):
        if arr[i, j] in ".SE":
            g.add_node((i, j))

def is_valid(arr, n):
    return 0 <= n[0] < arr.shape[0] and 0 <= n[1] < arr.shape[1] 

def is_path(arr, n):
    return is_valid(arr, n) and arr[*n] in ".SE"

def is_wall(arr, n):
    return is_valid(arr,  n) and arr[*n] == "#"


cand_cheats = set()
for i, j in list(g.nodes):
    for d in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
        n = (i + d[0], j + d[1])
        if is_path(arr, n):
            g.add_edge((i, j), n)

        n2 = (n[0] + d[0], n[1] + d[1])
        if is_wall(arr, n) and is_path(arr, n2):
            cand_cheats.add(tuple(sorted(((i,j),  n2))))


print(len(cand_cheats))


t = np.where(arr == "S")
s = (int(t[0][0]), int(t[1][0]))
t = np.where(arr == "E")
e = (int(t[0][0]), int(t[1][0]))

L = nx.shortest_path_length(g, s, e)


ds = nx.shortest_path_length(g, source=s)
de = nx.shortest_path_length(g, source=e)

cheats = [L - min(ds[c[0]]+ de[c[1]], ds[c[1]] + de[c[0]])-2 for c in cand_cheats]

from collections import Counter


s = 0
for k,v in sorted(Counter(cheats).items()):
    if k >= 100:
        s += v
    print(f"{v} cheats with {k} steps")

print(s)